◆役割
あなたはpythonのプログラマのスペシャリスト
LLMの実装に習熟している

◆仕様
顧客情報を要約させるチャットを作りたい
RAGは使わない

プロンプト案を複数作成する
DSPYを使ってプロンプトを最適化する
顧客情報を取り込む（サンプルデータでよい）
LLMに連携し、要約結果を得る
要約結果が簡潔であるか、わかりやすいか、一貫性があるかをLLMasaJudgeでテストする
LLMの回答は再現性がないため、tempertureを0にしたうえで、3回施行し、その結果の平均値を評価する
評価の関数はDSpyのevaluateでカスタムの評価関数をさk巣栄する
施行の結果をMLFlowで

OpenAIのAPIを利用
モデル：gpt4-o-mini
APIキー：XXXXX（別途設定）

まずはノートブックで実行したい
動作が確認できてからクラス化、関数化をするので、それを意識した処理に分けてほしい
